# 🧪 Tarea: Rendimiento con listas de Python vs NumPy

## 🎯 Objetivos
- Entender la **ventaja de la vectorización** frente a bucles en Python puro.
- Medir tiempos de ejecución de **operaciones estadísticas** (suma, media, mediana, desviación estándar) sobre datos grandes.
- Documentar resultados y extraer **conclusiones**.

## 🧰 Entorno
- Ejecuta este notebook en **Jupyter/Colab** o local con Python 3.12+.
- Necesitas `numpy` (y opcionalmente `pandas` para presentar tablas más bonitas).

---

## 1) Preparación
Define tamaños de datos y comprueba las versiones de Python y NumPy.

In [1]:
import sys, platform, math, statistics, random
from time import perf_counter

try:
    import numpy as np
except Exception as e:
    raise SystemExit("NumPy no está instalado. Instálalo con `pip install numpy`.\n" + str(e))

print("Python:", sys.version.split()[0])
print("Plataforma:", platform.platform())
print("NumPy:", np.__version__)

Python: 3.10.11
Plataforma: Windows-10-10.0.26100-SP0
NumPy: 1.26.4


## 2) Utilidades de medición
Incluye una función de **bench** con *warm-up* y repeticiones. No edites esta celda salvo que sepas lo que haces.

### Explicación de la función `bench`

La función `bench` permite medir el **tiempo medio de ejecución** de una función cualquiera (`func`) sobre unos argumentos dados. Realiza una ejecución de "calentamiento" (*warm-up*) para evitar efectos de inicialización, y luego repite la medición varias veces (por defecto 3), devolviendo el promedio de los tiempos obtenidos. Es útil para comparar el rendimiento de distintas implementaciones de una misma operación.

- **Parámetros principales:**
  - `func`: función a medir.
  - `*args`, `**kwargs`: argumentos para la función.
  - `repeats`: número de repeticiones.
  - `warmup`: si se realiza una ejecución previa de calentamiento.

Devuelve el tiempo medio en segundos.

In [4]:
def bench(func, *args, repeats=3, warmup=True, **kwargs):
    """Devuelve el tiempo medio (seg) de ejecutar func(*args, **kwargs)."""
    if warmup:
        func(*args, **kwargs)
    times = []
    for _ in range(repeats):
        t0 = perf_counter()
        func(*args, **kwargs)
        times.append(perf_counter() - t0)
    return sum(times) / len(times)

def gen_list(N, seed=42):
    random.seed(seed)
    return [random.random() for _ in range(N)]

## 3) Funciones en Python puro y con NumPy
⚠️ **No mezcles** generación de datos con la medición. Crea el `np.array` **una vez** por tamaño.

In [3]:
# Python puro
def py_sum(x):      return sum(x)
def py_mean(x):     return sum(x) / len(x)
def py_median(x):   return statistics.median(x)
def py_std(x):      return statistics.pstdev(x)  # desviación estándar poblacional

# NumPy
def np_sum(a):      return np.sum(a)
def np_mean(a):     return np.mean(a)
def np_median(a):   return np.median(a)
def np_std(a):      return np.std(a)  # ddof=0 (poblacional)

### Explicación de las funciones estadísticas (Python puro y NumPy)

Estas funciones implementan operaciones estadísticas básicas sobre una colección de datos, tanto usando Python estándar como la librería NumPy:

- **Python puro:**
  - `py_sum(x)`: suma todos los elementos de la lista `x`.
  - `py_mean(x)`: calcula la media aritmética de la lista `x`.
  - `py_median(x)`: obtiene la mediana de la lista `x`.
  - `py_std(x)`: calcula la desviación estándar poblacional de la lista `x`.

- **NumPy:**
  - `np_sum(a)`: suma todos los elementos del array `a`.
  - `np_mean(a)`: calcula la media aritmética del array `a`.
  - `np_median(a)`: obtiene la mediana del array `a`.
  - `np_std(a)`: calcula la desviación estándar poblacional del array `a`.

Las funciones de NumPy suelen ser mucho más rápidas en grandes volúmenes de datos gracias a la vectorización.

## 4) Ejecuta el experimento
Por defecto usaremos tamaños `N = 100k, 500k, 1M`. Si tu equipo es modesto, reduce los valores.

In [5]:
Ns = [100_000, 500_000, 1_000_000]  # ajusta si es necesario
repeats = 3

results = []
for N in Ns:
    L = gen_list(N)
    A = np.array(L, dtype=np.float64)
    row = {"N": N}

    # Python puro
    row["py_sum"]    = bench(py_sum,    L, repeats=repeats)
    row["py_mean"]   = bench(py_mean,   L, repeats=repeats)
    row["py_median"] = bench(py_median, L, repeats=repeats)
    row["py_std"]    = bench(py_std,    L, repeats=repeats)

    # NumPy
    row["np_sum"]    = bench(np_sum,    A, repeats=repeats)
    row["np_mean"]   = bench(np_mean,   A, repeats=repeats)
    row["np_median"] = bench(np_median, A, repeats=repeats)
    row["np_std"]    = bench(np_std,    A, repeats=repeats)

    results.append(row)

def fmt(s):
    try:
        return f"{s:.6f}"
    except Exception:
        return str(s)

print("N\tpy_sum\tnp_sum\tpy_mean\tnp_mean\tpy_median\tnp_median\tpy_std\tnp_std")
for r in results:
    print(f"{r['N']}\t{fmt(r['py_sum'])}\t{fmt(r['np_sum'])}\t{fmt(r['py_mean'])}\t{fmt(r['np_mean'])}\t"
          f"{fmt(r['py_median'])}\t{fmt(r['np_median'])}\t{fmt(r['py_std'])}\t{fmt(r['np_std'])}")

N	py_sum	np_sum	py_mean	np_mean	py_median	np_median	py_std	np_std
100000	0.000248	0.000030	0.000244	0.000031	0.010283	0.000936	0.095090	0.000100
500000	0.001338	0.000136	0.001361	0.000131	0.066547	0.004775	0.474739	0.000443
1000000	0.003323	0.000316	0.003979	0.000254	0.159266	0.008208	0.936564	0.000767


### Explicación del experimento de rendimiento

Esta celda ejecuta el experimento principal:

- Define varios tamaños de datos (`N`).
- Genera listas y arrays NumPy de números aleatorios.
- Mide el tiempo de ejecución de cada operación estadística (suma, media, mediana, desviación estándar) usando tanto Python puro como NumPy.
- Almacena los resultados en una lista de diccionarios.
- Presenta los resultados en formato de tabla para comparar el rendimiento.

Permite observar cómo escala el tiempo de cálculo al aumentar el tamaño de los datos y comparar la eficiencia entre Python puro y NumPy.

### (Opcional) Presentar la tabla con `pandas` y hacer gráfica simple

In [5]:
try:
    import pandas as pd
    df = pd.DataFrame(results)
    display(df)
except Exception:
    print("Pandas no está instalado; omitiendo visualización con DataFrame.")
    df = None

,N,py_sum,py_mean,py_median,py_std,np_sum,np_mean,np_median,np_std
0,100000,0.000950,0.000822,0.025019,0.078049,0.000055,0.000040,0.001181,0.000133
1,500000,0.004006,0.005281,0.247486,0.347542,0.000627,0.000279,0.008906,0.001868
2,1000000,0.009872,0.008892,0.567648,0.657810,0.000655,0.000523,0.016211,0.007557


### Explicación de la visualización con pandas

Esta celda intenta mostrar los resultados del experimento en una tabla usando la librería `pandas`. Si `pandas` está instalado, se crea un `DataFrame` y se visualiza para facilitar la comparación de los tiempos. Si no está disponible, se omite la visualización.

In [6]:
# Gráfica opcional (si pandas y matplotlib están disponibles)
try:
    import matplotlib.pyplot as plt
    if df is not None:
        metrics = ["sum", "mean", "median", "std"]
        for m in metrics:
            plt.figure()
            plt.plot(df["N"], df[f"py_{m}"], marker='o', label=f"Python {m}")
            plt.plot(df["N"], df[f"np_{m}"], marker='o', label=f"NumPy {m}")
            plt.xlabel("N")
            plt.ylabel("Tiempo (s)")
            plt.title(f"Rendimiento Python vs NumPy – {m}")
            plt.legend()
            plt.show()
except Exception:
    print("Matplotlib no está instalado; omitiendo gráficas.")

Matplotlib no está instalado; omitiendo gráficas.


### Explicación de la gráfica de rendimiento

Esta celda genera gráficas (si están disponibles `pandas` y `matplotlib`) que muestran el tiempo de ejecución de cada operación estadística para distintos tamaños de datos, comparando Python puro y NumPy. Las gráficas ayudan a visualizar de forma clara las diferencias de rendimiento.

## 5) (Opcional) Extensión con **JAX NumPy**
En Colab suele venir preinstalado. Si no, instala con `pip install jax jaxlib` (en CPU) o consulta la docu de JAX para GPU.

In [7]:
USE_JAX = True  # cambia a True si quieres probar JAX
if USE_JAX:
    try:
        import jax
        import jax.numpy as jnp
        print("JAX:", jax.__version__)
        print("JAX backend:", jax.default_backend())
        
        def jnp_sum(a):    return jnp.sum(a).block_until_ready()
        def jnp_mean(a):   return jnp.mean(a).block_until_ready()
        def jnp_median(a): return jnp.median(a).block_until_ready()
        def jnp_std(a):    return jnp.std(a).block_until_ready()

        results_jax = []
        for N in Ns:
            L = gen_list(N)
            A = np.array(L, dtype=np.float64)
            J = jnp.array(A)
            _ = jnp_sum(J)  # Warm-up compilación JIT

            row = {"N": N}
            row["jax_sum"]    = bench(jnp_sum,    J, repeats=repeats)
            row["jax_mean"]   = bench(jnp_mean,   J, repeats=repeats)
            row["jax_median"] = bench(jnp_median, J, repeats=repeats)
            row["jax_std"]    = bench(jnp_std,    J, repeats=repeats)
            results_jax.append(row)

        try:
            import pandas as pd
            display(pd.DataFrame(results_jax))
        except Exception:
            print(results_jax)
    except Exception as e:
        print("No se pudo usar JAX:", e)

No se pudo usar JAX: No module named 'jax'


### Explicación de la extensión con JAX NumPy

Esta celda permite repetir el experimento usando la librería JAX, que implementa operaciones similares a NumPy pero con compilación JIT y soporte para GPU/TPU. Si se activa `USE_JAX`, se mide el rendimiento de las operaciones estadísticas con JAX y se presentan los resultados.

## 6) Conclusión (rellenar por el alumno)
- ¿Qué operaciones mejoran más con NumPy y por qué?
- ¿Cómo escala el tiempo al crecer N?
- Si probaste JAX: ¿qué diferencia observaste respecto a NumPy en tu entorno?